## 🚀 Program 3

### 📋 Objective

##### Write a program to demonstrate the working of the decision tree based ID3 algorithm. Use an appropriate data set for building the decision tree and apply this knowledge to classify a new sample. 

In [10]:
# Import necessary libraries
import pandas as pd
import math
from collections import Counter

In [11]:
# Read the dataset
df = pd.read_csv("tennis.csv")

In [12]:
# Extract features and data from the DataFrame
features = list(df.columns[:-1])
data = df.values.tolist()

In [13]:
# Function to calculate the entropy of a dataset
def entropy(rows):
    labels = [row[-1] for row in rows]
    counts = Counter(labels)
    total = len(labels)
    return -sum((c/total)*math.log2(c/total) for c in counts.values())

In [14]:
# Function to calculate the information gain of a feature
def info_gain(rows, idx):
    total = entropy(rows)
    vals = set(row[idx] for row in rows)
    splits = [[row for row in rows if row[idx]==v] for v in vals]
    weighted = sum(len(s)/len(rows)*entropy(s) for s in splits)
    return total - weighted

In [15]:
# Function to build the decision tree using ID3 algorithm
def id3(rows, feats):
    labels = [row[-1] for row in rows]
    if labels.count(labels[0]) == len(labels): return labels[0]
    if not feats: return Counter(labels).most_common(1)[0][0]
    gains = [info_gain(rows, i) for i in range(len(feats))]
    best = gains.index(max(gains))
    tree = {feats[best]: {}}
    vals = set(row[best] for row in rows)
    for v in vals:
        sub = [row[:best]+row[best+1:] for row in rows if row[best]==v]
        sub_feats = feats[:best]+feats[best+1:]
        tree[feats[best]][v] = id3(sub, sub_feats)
    return tree

In [16]:
# Build the decision tree
def classify(tree, feats, sample):
    if not isinstance(tree, dict): return tree
    attr = next(iter(tree))
    idx = feats.index(attr)
    val = sample[idx]
    subtree = tree[attr].get(val, "Unknown")
    if isinstance(subtree, dict):
        return classify(subtree, feats, sample)
    return subtree

In [17]:
# Print the decision tree
print("Decision Tree:", id3(data, features))

Decision Tree: {'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [18]:
# Predict a sample
sample = ['Sunny', 'Cool', 'High', 'Strong']
print("Sample:", sample)
print("Predicted:", classify(id3(data, features), features, sample))

Sample: ['Sunny', 'Cool', 'High', 'Strong']
Predicted: No
